In [1]:
import sqlite3
import pandas as pd
import numpy as np

conn = sqlite3.connect('UMGC.db')
c = conn.cursor()

/var/folders/zy/3gdh0s5n2x38gj3cqxm_4n5r0000gn/T/ipykernel_6767/3364173113.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
def drop_view(table, db=c):
    db.execute('DROP VIEW IF EXISTS ' + table)

In [6]:
c.execute('select * from program_requirements where program_id=?', (10,))

In [7]:
row = c.fetchone()

In [8]:
row

(2, 10, 33, 12, 0, 75, 120)

In [9]:
conn.row_factory=sqlite3.Row
db = conn.cursor()

In [18]:
db.execute('select * from program_requirements where program_id=?', (10,))

In [19]:
rows = db.fetchall()

In [21]:
dict_rows = [dict(row) for row in rows]

In [24]:
len(dict_rows)

1

In [25]:
row = dict_rows[0]
row

{'id': 2,
 'program_id': 10,
 'major': 33,
 'related_ge': 12,
 'related_elective': 0,
 'remaining': 75,
 'total': 120}

In [5]:
drop_view('student_records_view_old')
create_view_query = '''
    CREATE VIEW student_records_view_old AS
    SELECT 
        a.seq as seq,
        a.student_info_id as student_info_id,
        a.name as name,
        a.credits as credits,
        a.type as type,
        a.completed as completed,
        a.period as period,
        a.session as session,
        a.prerequisite as prerequisite,
        IFNULL(b.title, '') AS title,
        IFNULL(b.description, '') AS description,
        IFNULL(b.prerequisites, '') as prereq_full
    FROM 
        student_progress a
    LEFT JOIN 
        classes b
    ON 
        a.name = b.name
'''
c.execute(create_view_query)
conn.commit()

In [5]:
drop_view('student_records_view')
create_view_query = '''
    CREATE VIEW student_records_view_old AS
    SELECT 
        a.seq as seq,
        a.student_info_id as student_id,
        a.name as name,
        a.credits as credits,
        a.type as type,
        a.completed as completed,
        a.period as period,
        a.session as session,
        a.prerequisite as prerequisite,
        IFNULL(b.title, '') AS title,
        IFNULL(b.description, '') AS description,
        IFNULL(b.prerequisites, '') as prereq_full
    FROM 
        student_progress a
    LEFT JOIN 
        classes b
    ON 
        a.name = b.name
'''
c.execute(create_view_query)
conn.commit()

In [3]:
c.execute("select * from student_records_view_old")

OperationalError: no such column: student_info_id

In [ ]:
w

In [ ]:
def single_query(query, cursor):
    # convenience function for sqlite3 db queries that return one value
    cursor.execute(query)
    q_result = cursor.fetchone()
    if q_result is not None:
        result = q_result[0]
    else:
        result = None

    return result

In [ ]:
def prepare_d3_data(df, start_term='SPRING 2024'):

    def set_colors(row):
        if row['type'] == 'general':
            return pd.Series(['green', 'white'])
        elif row['type'] == 'major':
            return pd.Series(['blue', 'white'])
        elif row['type'] == 'required':
            return pd.Series(['red', 'white'])
        elif row['type'] == 'elective':
            return pd.Series(['yellow', 'black'])
        else:
            return pd.Series(['white', 'black'])  # default colors

    def generate_header_data(start_semester, num_periods, data_df = df):
        seasons = ['WINTER', 'SPRING', 'SUMMER', 'FALL']
        semester_data = []
        start_season, start_year = start_semester.split(' ')
        start_year = int(start_year)
        season_index = seasons.index(start_season)
        year = start_year
        period = 0

        while period < num_periods:
            for j in range(season_index, len(seasons)):
                semester_data.append(f'{seasons[j]} {year}')
                period += 1

                # Break the loop when i equals num_periods
                if period == num_periods:
                    break

            # Reset the season index to start from 'WINTER' for the next year
            season_index = 0
            year += 1

        df = pd.DataFrame(semester_data, columns=['term'])
        df['width'] = df['term'].apply(lambda x: 190 if 'SUMMER' in x else 260)
        df['offset'] = df['term'].apply(lambda x: 2 if 'SUMMER' in x else 3)
        df['fontsize'] = '14px'
        df['description'] = ''
        df['space'] = 40
        df['xpos'] = df['width'] + df['space']

        x0 = 10
        # Calculate the cumulative sum of 'xpos'
        df['x'] = df['xpos'].cumsum()
        df['x'] = df['x'].shift(1)
        df.loc[0, 'x'] = 0
        df['x'] = df['x'] + x0
        df['y'] = 10
        df['color'] = 'lightgray'
        df['textcolor'] = 'black'
        df['period'] = np.arange(1, num_periods+1)

        df.drop
        # Sum credits per period and convert to a DataFrame
        total_credits = data_df.groupby('period')['credits'].sum().sort_index()
        total_credits_df = total_credits.reset_index()

        df = pd.merge(df, total_credits_df, on='period', how='inner')
        df['name'] = df['term']
        df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'

        return df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 
                   'fontsize', 'period', 'name', 'credits', 'description']]

    # Prepare data for the D3 figure

    max_period = max(df['period'])
    headers = generate_header_data(start_term, max_period)

    df['description'] = df['prerequisite']
    df['width'] = 120
    # Calculate 'x' column
    df = pd.merge(df, headers[['period','x']], on='period', how='left')
    df['x'] += 70*(df['session']-1)

    # Calculate 'y' column
    df = df.sort_values(by=['period', 'session', 'seq' ])
    df['y_row'] = df.groupby('period').cumcount() + 1
    df['y'] = 70 + 45 * (df['y_row'] - 1)

    # Create rectangle colors
    df[['color', 'textcolor']] = df.apply(set_colors, axis=1)

    # Set text offset multiplier to 1 and text fontsize
    df['offset'] = 1
    df['fontsize'] = '12px'
    df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'
    
    df = df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 'fontsize', 'period', 'session', 'type', 'name', 'credits', 'description']]

    return df, headers

# General Education

In [ ]:
def single_query(query, cursor):
    # convenience function for sqlite3 db queries that return one value
    cursor.execute(query)
    q_result = cursor.fetchone()
    if q_result is not None:
        result = q_result[0]
    else:
        result = None

    return result


In [ ]:
student_name_query = '''
    SELECT users.firstname || ' ' || users.lastname AS 'name'
        FROM users 
        INNER JOIN student_info ON users.id = student_info.user_id 
        WHERE student_info.id = {}
'''

In [ ]:
student_info_id = 3
student_name = single_query(student_name_query.format(student_info_id), cur)    
student_progress_query = 'SELECT * FROM student_progress WHERE student_info_id={}'.format(student_info_id)
df_raw = pd.read_sql_query(student_progress_query, conn)

In [ ]:
course_summary = df_raw.groupby(['type','completed']).agg(
    n=('credits', 'count'),
    total=('credits', 'sum')
).reset_index()
course_summary

In [ ]:
df = df_raw[df_raw['completed'] == 0].copy() # create a separate dataframe

In [ ]:
import utils
# pick up start_term from the form
start_term = 'SPRING 2024'

# may need to rewrite this for later
df_d3, headers = utils.prepare_d3_data(df, start_term.upper())


In [ ]:
df_filtered = df_filtered.merge(headers[['period', 'name']].rename(columns={'name': 'term'}), on='period', how='left')

In [ ]:
records = df0.to_dict('records')


In [ ]:
course_summary = df0.groupby(['type','completed']).agg(
    n=('credits', 'count'),
    total=('credits', 'sum')
).reset_index()
summary

records[0]

## Dev 

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('UMGC.db')

In [ ]:
c = conn.cursor()
c.execute('''
    CREATE TABLE classes (
        name TEXT,
        title TEXT,
        credit TEXT,
        description TEXT,
        prerequisites TEXT,
        recommended TEXT,
        warnings TEXT,
        substitutions TEXT,
        pre TEXT,
        pre_credits TEXT,
        pre_notes TEXT,
        done INTEGER
    )
''')


In [ ]:
# Insert data into the table

for class_name, class_info in classes.items():
    c.execute('''
        INSERT INTO classes VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        class_info['name'],
        class_info['title'],
        class_info['credit'],
        class_info['description'],
        class_info['prerequisites'],
        class_info['recommended'],
        class_info['warnings'],
        class_info['substitutions'],
        class_info['pre'],
        class_info['pre_credits'],
        class_info['pre_notes'],
        int(class_info['done'])
    ))


In [ ]:
conn.commit()
conn.close()

In [ ]:
json_data = df.to_json(orient='records')


In [ ]:
data_dict = df.to_dict(orient='records')


In [ ]:
data_json_new = [
    { "seq":  1, "name": "PACE 111", "credits": 3, "type": "general",  "period":  1, "session": 1, "prerequisite": ""                                },
    { "seq":  2, "name": "LIBS 150", "credits": 1, "type": "general",  "period":  1, "session": 2, "prerequisite": ""                                },
    { "seq":  3, "name": "WRTG 111", "credits": 3, "type": "general",  "period":  1, "session": 3, "prerequisite": ""                                },
    { "seq":  4, "name": "WRTG 112", "credits": 3, "type": "general",  "period":  2, "session": 1, "prerequisite": ""                                },
    { "seq":  5, "name": "NUTR 100", "credits": 3, "type": "general",  "period":  2, "session": 2, "prerequisite": ""                                },
    { "seq":  6, "name": "BMGT 110", "credits": 3, "type": "major",    "period":  2, "session": 3, "prerequisite": ""                                },
    { "seq":  7, "name": "SPCH 100", "credits": 3, "type": "general",  "period":  3, "session": 1, "prerequisite": ""                                },
    { "seq":  8, "name": "STAT 200", "credits": 3, "type": "required", "period":  3, "session": 2, "prerequisite": ""                                },
    { "seq":  9, "name": "IFSM 300", "credits": 3, "type": "required", "period":  4, "session": 1, "prerequisite": ""                                },
    { "seq": 10, "name": "ACCT 220", "credits": 3, "type": "major",    "period":  4, "session": 1, "prerequisite": ""                                }, 
    { "seq": 11, "name": "HUMN 100", "credits": 3, "type": "general",  "period":  4, "session": 2, "prerequisite": ""                                }, 
    { "seq": 12, "name": "BIOL 103", "credits": 4, "type": "general",  "period":  5, "session": 1, "prerequisite": ""                                }, 
    { "seq": 13, "name": "ECON 201", "credits": 3, "type": "required", "period":  4, "session": 3, "prerequisite": ""                                }, 
    { "seq": 14, "name": "ARTH 334", "credits": 3, "type": "general",  "period":  5, "session": 2, "prerequisite": ""                                },
    { "seq": 15, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  6, "session": 1, "prerequisite": ""                                }, 
    { "seq": 16, "name": "ECON 203", "credits": 3, "type": "required", "period":  6, "session": 2, "prerequisite": ""                                }, 
    { "seq": 17, "name": "ACCT 221", "credits": 3, "type": "major",    "period":  6, "session": 3, "prerequisite": "ACCT 220"                        }, 
    { "seq": 18, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  7, "session": 1, "prerequisite": ""                                }, 
    { "seq": 19, "name": "BMGT 364", "credits": 3, "type": "major",    "period":  7, "session": 2, "prerequisite": ""                                }, 
    { "seq": 20, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  8, "session": 1, "prerequisite": ""                                }, 
    { "seq": 21, "name": "BMGT 365", "credits": 3, "type": "major",    "period":  8, "session": 2, "prerequisite": "BMGT 364"                        }, 
    { "seq": 22, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  8, "session": 3, "prerequisite": ""                                }, 
    { "seq": 23, "name": "MRKT 310", "credits": 3, "type": "major",    "period":  8, "session": 1, "prerequisite": ""                                }, 
    { "seq": 24, "name": "WRTG 394", "credits": 3, "type": "general",  "period":  9, "session": 1, "prerequisite": "WRTG 112"                        }, 
    { "seq": 25, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  9, "session": 2, "prerequisite": ""                                }, 
    { "seq": 26, "name": "BMGT 380", "credits": 3, "type": "major",    "period":  9, "session": 3, "prerequisite": ""                                }, 
    { "seq": 27, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 10, "session": 1, "prerequisite": ""                                }, 
    { "seq": 28, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 10, "session": 2, "prerequisite": ""                                }, 
    { "seq": 29, "name": "HRMN 300", "credits": 3, "type": "major",    "period": 10, "session": 3, "prerequisite": ""                                },
    { "seq": 30, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 11, "session": 1, "prerequisite": ""                                }, 
    { "seq": 31, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 11, "session": 2, "prerequisite": ""                                }, 
    { "seq": 32, "name": "FINC 330", "credits": 3, "type": "major",    "period": 12, "session": 1, "prerequisite": "ACCT 221 & STAT 200"             }, 
    { "seq": 33, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 12, "session": 2, "prerequisite": ""                                }, 
    { "seq": 34, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 12, "session": 3, "prerequisite": ""                                }, 
    { "seq": 35, "name": "BMGT 496", "credits": 3, "type": "major",    "period": 12, "session": 1, "prerequisite": ""                                }, 
    { "seq": 36, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 13, "session": 1, "prerequisite": ""                                }, 
    { "seq": 37, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 13, "session": 2, "prerequisite": ""                                }, 
    { "seq": 38, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 13, "session": 3, "prerequisite": ""                                }, 
    { "seq": 39, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 14, "session": 1, "prerequisite": ""                                }, 
    { "seq": 40, "name": "BMGT 495", "credits": 3, "type": "major",    "period": 14, "session": 2, "prerequisite": "BMGT 365 & MRKT 310 & FINC 330"  }, 
    { "seq": 41, "name": "CAPSTONE", "credits": 1, "type": "elective", "period": 14, "session": 3, "prerequisite": "FINC 330"                        }, 
]


## Updated Python Code

In [ ]:
import pandas as pd
import numpy as np

def generate_header_data(start_semester, num_periods, data_df = df):
    seasons = ['WINTER', 'SPRING', 'SUMMER', 'FALL']
    semester_data = []
    start_season, start_year = start_semester.split(' ')
    start_year = int(start_year)
    season_index = seasons.index(start_season)
    year = start_year
    period = 0

    while period < num_periods:
        for j in range(season_index, len(seasons)):
            semester_data.append(f'{seasons[j]} {year}')
            period += 1

            # Break the loop when i equals num_periods
            if period == num_periods:
                break

        # Reset the season index to start from 'WINTER' for the next year
        season_index = 0
        year += 1

    df = pd.DataFrame(semester_data, columns=['term'])
    df['width'] = df['term'].apply(lambda x: 190 if 'SUMMER' in x else 260)
    df['offset'] = df['term'].apply(lambda x: 2 if 'SUMMER' in x else 3)
    df['fontsize'] = '14px'
    df['description'] = ''
    df['space'] = 40
    df['xpos'] = df['width'] + df['space']

    x0 = 10
    # Calculate the cumulative sum of 'xpos'
    df['x'] = df['xpos'].cumsum()
    df['x'] = df['x'].shift(1)
    df.loc[0, 'x'] = 0
    df['x'] = df['x'] + x0
    df['y'] = 10
    df['color'] = 'lightgray'
    df['textcolor'] = 'black'
    df['period'] = np.arange(1, num_periods+1)

    df.drop
    # Sum credits per period and convert to a DataFrame
    total_credits = data_df.groupby('period')['credits'].sum().sort_index()
    total_credits_df = total_credits.reset_index()

    df = pd.merge(df, total_credits_df, on='period', how='inner')
    df['name'] = df['term']
    df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'

    return df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 'fontsize', 'period', 'name', 'credits', 'description']]

def set_colors(row):
    if row['type'] == 'general':
        return pd.Series(['green', 'white'])
    elif row['type'] == 'major':
        return pd.Series(['blue', 'white'])
    elif row['type'] == 'required':
        return pd.Series(['red', 'white'])
    elif row['type'] == 'elective':
        return pd.Series(['yellow', 'black'])
    else:
        return pd.Series(['white', 'black'])  # default colors

def prepare_d3_data(df, start_term='SPRING 2024'):
    # Function to prepare the data to be fed into the D3 code
    max_period = max(df['period'])
    headers = generate_header_data(start_term, max_period)

    df['description'] = df['prerequisite']
    df['width'] = 120
    # Calculate 'x' column
    df = pd.merge(df, headers[['period','x']], on='period', how='left')
    df['x'] += 70*(df['session']-1)

    # Calculate 'y' column
    df = df.sort_values(by=['period', 'session', 'seq' ])
    df['y_row'] = df.groupby('period').cumcount() + 1
    df['y'] = 70 + 45 * (df['y_row'] - 1)

    # Create rectangle colors
    df[['color', 'textcolor']] = df.apply(set_colors, axis=1)

    # Set text offset multiplier to 1 and text fontsize
    df['offset'] = 1
    df['fontsize'] = '12px'
    df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'
    
    df = df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 'fontsize', 'period', 'session', 'type', 'name', 'credits', 'description']]

    return df, headers

#### Get data from the database instead of the json file

In [ ]:
def prepare_d3_data(df, start_term='SPRING 2024'):

    def set_colors(row):
        if row['type'] == 'general':
            return pd.Series(['green', 'white'])
        elif row['type'] == 'major':
            return pd.Series(['blue', 'white'])
        elif row['type'] == 'required':
            return pd.Series(['red', 'white'])
        elif row['type'] == 'elective':
            return pd.Series(['yellow', 'black'])
        else:
            return pd.Series(['white', 'black'])  # default colors

    def generate_header_data(start_semester, num_periods, data_df = df):
        seasons = ['WINTER', 'SPRING', 'SUMMER', 'FALL']
        semester_data = []
        start_season, start_year = start_semester.split(' ')
        start_year = int(start_year)
        season_index = seasons.index(start_season)
        year = start_year
        period = 0

        while period < num_periods:
            for j in range(season_index, len(seasons)):
                semester_data.append(f'{seasons[j]} {year}')
                period += 1

                # Break the loop when i equals num_periods
                if period == num_periods:
                    break

            # Reset the season index to start from 'WINTER' for the next year
            season_index = 0
            year += 1

        df = pd.DataFrame(semester_data, columns=['term'])
        df['width'] = df['term'].apply(lambda x: 190 if 'SUMMER' in x else 260)
        df['offset'] = df['term'].apply(lambda x: 2 if 'SUMMER' in x else 3)
        df['fontsize'] = '14px'
        df['description'] = ''
        df['space'] = 40
        df['xpos'] = df['width'] + df['space']

        x0 = 10
        # Calculate the cumulative sum of 'xpos'
        df['x'] = df['xpos'].cumsum()
        df['x'] = df['x'].shift(1)
        df.loc[0, 'x'] = 0
        df['x'] = df['x'] + x0
        df['y'] = 10
        df['color'] = 'lightgray'
        df['textcolor'] = 'black'
        df['period'] = np.arange(1, num_periods+1)

        df.drop
        # Sum credits per period and convert to a DataFrame
        total_credits = data_df.groupby('period')['credits'].sum().sort_index()
        total_credits_df = total_credits.reset_index()

        df = pd.merge(df, total_credits_df, on='period', how='inner')
        df['name'] = df['term']
        df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'

        return df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 
                   'fontsize', 'period', 'name', 'credits', 'description']]

    # Prepare data for the D3 figure
    
    max_period = max(df['period'])
    headers = generate_header_data(start_term, max_period)

    df['description'] = df['prerequisite']
    df['width'] = 120
    # Calculate 'x' column
    df = pd.merge(df, headers[['period','x']], on='period', how='left')
    df['x'] += 70*(df['session']-1)

    # Calculate 'y' column
    df = df.sort_values(by=['period', 'session', 'seq' ])
    df['y_row'] = df.groupby('period').cumcount() + 1
    df['y'] = 70 + 45 * (df['y_row'] - 1)

    # Create rectangle colors
    df[['color', 'textcolor']] = df.apply(set_colors, axis=1)

    # Set text offset multiplier to 1 and text fontsize
    df['offset'] = 1
    df['fontsize'] = '12px'
    df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'
    
    df = df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 'fontsize', 'period', 'session', 'type', 'name', 'credits', 'description']]

    return df, headers


In [ ]:
df = pd.DataFrame(data_json_new)

# for new student, set course completion to False
# will pick up completed classes in database and from transfer credits
# all code will have to ensure we are working on pending courses and filter those where complete='False'
#
# Will need to do something similar for lock=True when the semester is assigned and we do not want to move it
# 

#df['complete'] = False

In [ ]:
start_term = 'SPRING 2024'
df, headers = prepare_d3_data(df, start_term)

In [ ]:
'SUMMER 2024'.capitalize()

In [ ]:
terms_remaining = max(headers.period)
terms_remaining

In [ ]:
credits_next_term = headers.loc[headers['period'] == 1, 'credits'].values[0]
credits_next_term

In [ ]:
completion_date = headers.loc[headers['period'] == terms_remaining, 'name'].values[0].capitalize()
completion_date

In [ ]:
completion_date.upper()

In [ ]:
total_credits_remaining = df['credits'].sum()
total_credits_remaining

In [ ]:
headers = generate_header_data('SPRING 2024', 14)

df['width'] = 120
# Calculate 'x' column
df = pd.merge(df, headers[['period','x']], on='period', how='left')
df['x'] += 70*(df['session']-1)

# Calculate 'y' column
df = df.sort_values(by=['period', 'session', 'seq' ])
df['y_row'] = df.groupby('period').cumcount() + 1
df['y'] = 70 + 45 * (df['y_row'] - 1)

# Create rectangle colors
df[['color', 'textcolor']] = df.apply(set_colors, axis=1)

# Set text offset multiplier to 1 and text fontsize
df['offset'] = 1
df['fontsize'] = '12px'

df = df[['x', 'y', 'width', 'name', 'credits', 'color', 'textcolor', 'offset', 
         'fontsize', 'period', 'session']]

In [ ]:
df